In [1]:
import sys
!{sys.executable} -m pip install pandas_profiling 

     |████████████████████████████████| 261 kB 23.8 MB/s eta 0:00:01
     |████████████████████████████████| 102 kB 19.3 MB/s ta 0:00:01
     |████████████████████████████████| 3.1 MB 72.3 MB/s eta 0:00:01
     |████████████████████████████████| 10.2 MB 29.5 MB/s eta 0:00:01
     |████████████████████████████████| 675 kB 31.4 MB/s eta 0:00:01
     |████████████████████████████████| 812 kB 42.1 MB/s eta 0:00:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=b6f96b33e6eede3f34bcf55842ab1c613f49df2e9f085c03918984351cf801ca
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/fe/0b/4450b38bceb9ae43dd7d0f16e353566f30f5f4d59a58eca2ed
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=89ac7ef0c424101c9eda4cbc54dbf8ecde8ada307eef8430cdf43da7890ff4fd
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5e/77/4c/7291601777365dd7db8a3b309006122d515ee5ac086f2f5866
Successfully built htmlmin imagehash
  A

In [1]:
from datetime import datetime
from IPython.display import display
# %reload_ext autotime

import pandas as pd
import numpy as np
import re
import joblib

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

In [2]:
def basicDataExploration(df):
    print("General Stats::")
    print(df.info())

    print( 100*'-')


    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[1] )

    print( 100*'-')

    print("Column Names:")
    print ( df.columns.values.tolist() )

    print( 100*'-')
    #Explore for missing values
    print("Columns with Missing Values:",df.columns[df.isnull().any()].tolist())
    
    #print("\nUnique Values Exploration: ")
    #features = df.columns.values
    #print (100*'-')
    #for feature in features:
        #unique_length =  len(df[str(feature)].unique() )
        #print(str(unique_length)+' Unique values for' + ' ' + f'"{feature}"' +': {}'.format(df[str(feature)].unique()))
        #print (100*'-')
    
def display_all(df):
    with pd.option_context("display.max_rows", 1000,"display.max_columns", 1000): 
        display(df)
        
def null_summary( df ):
    # Number of null values in each feature
    null = df.isnull().sum()
    # Select those columns that contains null values
    null_counts = pd.DataFrame ( null.loc[ null> float(0) ] , columns =["Frequency"] )
    # Get the percentage of missing data for those features with missing values
    null_normalized = \
    null_counts.apply( lambda x: x/(df.shape[0] ) ).rename( columns={"Frequency":"Percentage"})
    # obtain a [Frequency, Percentage] dataframe to visualise the missing data
    null_df = pd.concat( [null_counts, null_normalized] , axis=1).sort_values(by="Percentage")
   
    null_df = null_df.style.background_gradient(cmap='Reds')

    return null_df

def my_value_counts( data, col):
    df= pd.concat( [pd.DataFrame(
        data[col].value_counts(dropna=False) ).rename(
    columns={col:"Frequency"}) 
                ,pd.DataFrame(
                data[col].value_counts(dropna=False, normalize = True)).rename(
                columns={col:"Proportion"})]
              ,axis=1 )
    
    df = df.append(
        pd.DataFrame(
        {"Frequency":[df["Frequency"].sum()],
         "Proportion":[df["Proportion"].sum()]},
        index=['Total'])
    )
    
    df = df.round(decimals=4)
    df = df.style.background_gradient(cmap='Blues')
    
    return df 

In [3]:
#df_session = pd.read_csv('Raw_Data/session_hits.csv')
df_session_small = pd.read_csv('repeat_sessions.csv')

In [5]:
print('Number of repeat sessions: {}'.format(len(df_session_small)))

print('Number of return visitors: {}'.format(df_session_small['full_visitor_id'].nunique()/
                                             df_session['full_visitor_id'].nunique()))

Number of repeat sessions: 1215047
Number of return visitors: 0.35091823464548444


In [11]:
df_quotation = pd.read_csv('repeat_quotation.csv')
df_quotation

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,12,13,14,15,16,19,20,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,full_visitor_id,client_id,quotation_id,total_sessions_count,max_visit_number,first_session_date,latest_session_date,first_session_date_time,latest_session_date_time,avg_session_quality_dim,...,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator,quote_applicant_nric,first_policy_issue_date,scv_yor_at_quote_created,scv_first_product_type,scv_issued_nfc_policies,scv_existing_customer_flag
0,1.000282e+18,GA1.3.232896385.1622111081,UGXY4911826Q70Q,1,5,2021-08-20,2021-08-20,2021-08-20 14:26:37,2021-08-20 14:26:37,12,...,Cancer,2021-08-20 22:29:26.27,0.0,0.0,S7071699B,NaN,0,NaN,0,0
1,1.001015e+18,GA1.3.233066960.1624073760,EEDK4734606JMWQ,1,1,2021-06-19,2021-06-19,2021-06-19 03:36:00,2021-06-19 03:36:00,3,...,Motor,2021-06-19 11:36:53.91,0.0,0.0,NaN,NaN,0,NaN,0,0
2,1.001640e+18,GA1.3.233212497.1609767885,0AE54367224YFCF,1,2,2021-01-07,2021-01-07,2021-01-07 14:00:41,2021-01-07 14:00:41,7,...,MCycle,2021-01-07 22:01:41.893,1.0,0.0,NaN,NaN,0,NaN,0,0
3,1.003436e+18,GA1.3.233630727.1631075762,DE7G4666473RYUN,1,2,2021-09-30,2021-09-30,2021-09-30 08:24:35,2021-09-30 08:24:35,2,...,IH,2021-05-21 18:36:29.21,0.0,0.0,NaN,NaN,0,NaN,0,0
4,1.003507e+18,GA1.3.233647182.1611304081,23BO44028797Q7L,1,1,2021-01-22,2021-01-22,2021-01-22 08:28:02,2021-01-22 08:28:02,77,...,HDBFire,2021-01-22 16:31:09.423,0.0,1.0,S7712836J,2021-01-22 16:55:03.937,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722492,9.214535e+18,GA1.3.2145426093.1625889739,NaN,1,1,2021-07-10,2021-07-10,2021-07-10 04:02:19,2021-07-10 04:02:19,7,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
722493,9.239602e+17,GA1.3.215126256.1618034445,NaN,2,2,2021-04-10,2021-04-10,2021-04-10 06:00:45,2021-04-10 06:45:06,26,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
722494,9.339511e+17,GA1.3.217452426.1614700049,NaN,4,4,2021-03-02,2021-03-03,2021-03-02 15:47:28,2021-03-03 03:35:19,4,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
722495,9.531720e+17,GA1.3.221927659.1628425573,NaN,1,1,2021-08-08,2021-08-08,2021-08-08 12:26:13,2021-08-08 12:26:13,52,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0


In [8]:
df_home_small = pd.read_csv('Raw_Data/home_visitor_hits.csv')

,Unnamed: 0,date,fullvisitorid,clientid,visitnumber,uniquevisitid,visitstarttime,visitdate,channelgrouping,source,...,minute,isentrance,isexit,pagepath,pagepathlevel1,pagepathlevel2,pagepathlevel3,pagepathlevel4,pagetitle,type
0,0,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),...,15,NaN,t,www.fwd.com.sg/fire-insurance/online-quote/tha...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/thankyou,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
1,1,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),...,15,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/pro...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/processing/375342214/D0U44746667QABK,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2,2,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),...,3,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/sum...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/summary,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
3,3,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),...,57,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/app...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/application,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
4,4,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),...,57,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/NoA...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/NoAdditions,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2986170,2986170,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),...,8,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/qui...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/quickQuestions,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2986171,2986171,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),...,8,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2986172,2986172,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),...,8,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2986173,2986173,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),...,57,t,NaN,www.fwd.com.sg/fire-insurance/online-quote/qui...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/quickQuestions,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE


In [12]:
df_home_small = df_home_small.drop('Unnamed: 0',1)

### Converting Visitor_ID to Int

In [4]:
df_session_small = df_session_small.sort_values(['full_visitor_id','visit_number'])
df_session_small = df_session_small.reset_index()
df_session_small = df_session_small.drop('index',1)
df_session_small['full_visitor_id'] = df_session_small['full_visitor_id'].astype(int)
df_session_small

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
0,701696772471,GA1.3.163.1617103223,701696772471-1617103223,NaN,1,2021-03-30,2021-03-30 11:20:23,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/ren...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
1,701696772471,GA1.3.163.1617103223,701696772471-1617183010,NaN,2,2021-03-31,2021-03-31 09:30:10,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/portal/,Maid Insurance Online Quote | FWD Singapore,FWD Online Login Portal | FWD Singapore,NaN,NaN,NaN,NaN
2,701696772471,GA1.3.163.1617103223,701696772471-1617183311,NaN,3,2021-03-31,2021-03-31 09:35:11,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
3,701696772471,GA1.3.163.1617103223,701696772471-1617183736,NaN,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
4,701696772471,GA1.3.163.1617103223,701696772471-1617183736,8O3Z1JIIZ,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,Maid,2021-03-30 17:21:03.247,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215042,9223329934920336384,GA1.3.2147473845.1629963161,9223329934920336281-1629963160,NaN,1,2021-08-26,2021-08-26 07:32:40,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/home-insurance/renewal.html?id=...,www.fwd.com.sg/home-insurance/online-quote/dem...,Home Insurance Online Quote | FWD Singapore,Home Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
1215043,9223352981715395584,GA1.3.2147479211.1630512433,9223352981715395889-1630512431,NaN,1,2021-09-02,2021-09-01 16:07:11,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/car-insurance/,www.fwd.com.sg/car-insurance/comprehensive-mot...,Buy Car Insurance Plan Online | FWD Singapore,Car Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
1215044,9223352981715395584,GA1.3.2147479211.1630512433,9223352981715395889-1630512431,FCTU49414032YIH,1,2021-09-02,2021-09-01 16:07:11,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/car-insurance/,www.fwd.com.sg/car-insurance/comprehensive-mot...,Buy Car Insurance Plan Online | FWD Singapore,Car Insurance Online Quote | FWD Singapore,Motor,2021-09-02 00:09:06.21,0.0,0.0
1215045,-9223372036854775808,GA1.3.40df6747-c6e4-47a9-970a-bf5d8890634a,13681945308945359288-1617108480,NaN,1,2021-03-30,2021-03-30 12:48:00,Paid Search,google,cpc,...,0,0,www.fwd.com.sg.googleweblight.com/stroke-insur...,www.fwd.com.sg.googleweblight.com/stroke-insur...,NaN,NaN,NaN,NaN,NaN,NaN


### Basic Exploration of Session

In [6]:
basicDataExploration(df_session_small)

General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215047 entries, 0 to 1215046
Data columns (total 36 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   full_visitor_id              1215047 non-null  int64  
 1   client_id                    1215047 non-null  object 
 2   session_id                   1215047 non-null  object 
 3   quotation_id                 383995 non-null   object 
 4   visit_number                 1215047 non-null  int64  
 5   session_date                 1215047 non-null  object 
 6   session_date_time            1215047 non-null  object 
 7   channelgrouping              1215047 non-null  object 
 8   source                       1215047 non-null  object 
 9   medium                       1215047 non-null  object 
 10  keyword                      1215047 non-null  object 
 11  adcontent                    1215047 non-null  object 
 12  referralpath              

In [34]:
null_summary(df_session_small)

TypeError: null_summary() takes 1 positional argument but 2 were given

In [40]:
my_value_counts(df_session_small, 'session_id.1')

In [39]:
my_value_counts(df_session_small, 'session_quality_dim')

,Frequency,Proportion
1,165782,0.213800
3,50964,0.065700
4,41848,0.054000
5,36820,0.047500
2,36802,0.047500
6,30909,0.039900
7,25665,0.033100
8,19918,0.025700
9,15543,0.020000
10,15043,0.019400


In [10]:
my_value_counts(df_session_small, 'product_type')

,Frequency,Proportion
nan,850971,0.700400
Motor,111071,0.091400
Home,57728,0.047500
Maid,49598,0.040800
MCycle,29228,0.024100
HDBFire,28605,0.023500
Travel,22966,0.018900
PA,21934,0.018100
Big 3 Critical Illness,20665,0.017000
IH,9315,0.007700


In [11]:
my_value_counts(df_session_small, 'entrance_page_title')

,Frequency,Proportion
"Online Insurance Company, Insurance Claims | FWD Singapore",238690,0.196400
Buy Car Insurance Plan Online | FWD Singapore,144712,0.119100
HDB Fire Insurance Plans Online | FWD Singapore,78371,0.064500
Car Insurance Online Quote | FWD Singapore,74969,0.061700
Buy Maid Insurance Plan Online | FWD Singapore,67614,0.055600
Buy Motorcycle Insurance Plan Online | FWD Singapore,58102,0.047800
Buy Term Life Insurance Online | FWD Singapore,54002,0.044400
Buy Home Insurance Online | FWD Singapore,52034,0.042800
Home Insurance Online Quote | FWD Singapore,40465,0.033300
nan,35238,0.029000


In [17]:
df_session_small.loc[df_session_small['entrance_page_title'].isnull()]

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
158,785091590619217,GA1.3.182793.1633681489,785091590619217-1633681490,NaN,1,2021-10-08,2021-10-08 08:24:50,Direct,(direct),(none),...,0,0,www.fwd.com.sg/error-page/404,www.fwd.com.sg/home-insurance/,NaN,Buy Home Insurance Online | FWD Singapore,NaN,NaN,NaN,NaN
186,1012389843896307,GA1.3.235715.1627719667,1012389843896307-1628644703,NaN,3,2021-08-11,2021-08-11 01:18:23,(Other),Renewal,EDM,...,2,0,www.fwd.com.sg/personal-accident-insurance/ren...,www.fwd.com.sg/personal-accident-renewal-insur...,NaN,NaN,NaN,NaN,NaN,NaN
187,1012389843896307,GA1.3.235715.1627719667,1012389843896307-1628644705,NaN,4,2021-08-11,2021-08-11 01:18:25,Referral,link.sg.fwd.com,referral,...,6,0,www.fwd.com.sg/personal-accident-renewal-insur...,www.fwd.com.sg/portal/,NaN,FWD Online Login Portal | FWD Singapore,NaN,NaN,NaN,NaN
240,1218612696524046,GA1.3.283730.1625629966,1218612696524046-1626865323,NaN,3,2021-07-21,2021-07-21 11:02:03,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/error-page/404,www.fwd.com.sg/error-page/404,NaN,NaN,NaN,NaN,NaN,NaN
328,1993961663615368,GA1.3.464255.1621610889,1993961663615369-1621655050,NaN,2,2021-05-22,2021-05-22 03:44:10,Direct,(direct),(none),...,0,0,www.fwd.com.sg/error-page/500,www.fwd.com.sg/error-page/500,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214992,9223009757994330112,GA1.3.2147399298.1630972175,9223009757994330383-1630972175,9FYF4955975LJVQ,1,2021-09-07,2021-09-06 23:49:35,Direct,(direct),(none),...,0,0,www.fwd.com.sg/error-page/404,www.fwd.com.sg/referrals,NaN,Insurance Promotions: Refer a Friend | FWD Sin...,Home,2021-09-07 07:51:31.467,0.0,1.0
1214994,9223017888362882048,GA1.3.2147401191.1626432640,9223017888362882176-1626449793,NaN,2,2021-07-16,2021-07-16 15:36:33,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/error-page/500,www.fwd.com.sg/error-page/500,NaN,NaN,NaN,NaN,NaN,NaN
1214995,9223017888362882048,GA1.3.2147401191.1626432640,9223017888362882176-1626658590,NaN,3,2021-07-19,2021-07-19 01:36:30,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/error-page/500,www.fwd.com.sg/error-page/500,NaN,NaN,NaN,NaN,NaN,NaN
1215045,-9223372036854775808,GA1.3.40df6747-c6e4-47a9-970a-bf5d8890634a,13681945308945359288-1617108480,NaN,1,2021-03-30,2021-03-30 12:48:00,Paid Search,google,cpc,...,0,0,www.fwd.com.sg.googleweblight.com/stroke-insur...,www.fwd.com.sg.googleweblight.com/stroke-insur...,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
my_value_counts(df_session_small, 'exit_page_title')

,Frequency,Proportion
Car Insurance Online Quote | FWD Singapore,193824,0.159500
"Online Insurance Company, Insurance Claims | FWD Singapore",110351,0.090800
HDB Fire Plan - Instant Quote Online | FWD Singapore,93921,0.077300
Maid Insurance Online Quote | FWD Singapore,82387,0.067800
Home Insurance Online Quote | FWD Singapore,82226,0.067700
Get an Online Motorcycle Insurance Quote | FWD Singapore,75352,0.062000
Buy Car Insurance Plan Online | FWD Singapore,45148,0.037200
Buy Term Life Insurance Online | FWD Singapore,40387,0.033200
FWD,37161,0.030600
FWD Online Login Portal | FWD Singapore,36594,0.030100


In [19]:
df_session_small.loc[df_session_small['exit_page_title'].isnull()]

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
186,1012389843896307,GA1.3.235715.1627719667,1012389843896307-1628644703,NaN,3,2021-08-11,2021-08-11 01:18:23,(Other),Renewal,EDM,...,2,0,www.fwd.com.sg/personal-accident-insurance/ren...,www.fwd.com.sg/personal-accident-renewal-insur...,NaN,NaN,NaN,NaN,NaN,NaN
240,1218612696524046,GA1.3.283730.1625629966,1218612696524046-1626865323,NaN,3,2021-07-21,2021-07-21 11:02:03,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/error-page/404,www.fwd.com.sg/error-page/404,NaN,NaN,NaN,NaN,NaN,NaN
320,1899386485110222,GA1.3.442235.1622963662,1899386485110222-1622963661,NaN,1,2021-06-06,2021-06-06 07:14:21,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/car-insurance/renewal.html?id=1...,www.fwd.com.sg/app/SGApp,Car Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,NaN
328,1993961663615368,GA1.3.464255.1621610889,1993961663615369-1621655050,NaN,2,2021-05-22,2021-05-22 03:44:10,Direct,(direct),(none),...,0,0,www.fwd.com.sg/error-page/500,www.fwd.com.sg/error-page/500,NaN,NaN,NaN,NaN,NaN,NaN
378,2381745682206586,GA1.3.554543.1632980858,2381745682206586-1633417577,NaN,5,2021-10-05,2021-10-05 07:06:17,Direct,(direct),(none),...,0,0,www.fwd.com.sg/app/SGApp,www.fwd.com.sg/app/SGApp,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215004,9223027354476510208,GA1.3.2147403395.1632140691,9223027354476510611-1632876890,NaN,4,2021-09-29,2021-09-29 00:54:50,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/car-insurance/renewal.html?id=b...,www.fwd.com.sg/app/SGApp,Car Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,NaN
1215016,9223133706373663744,GA1.3.2147428157.1549110685,9223133706373664157-1611759332,NaN,3,2021-01-27,2021-01-27 14:55:32,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/error-page/500,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,NaN
1215017,9223133706373663744,GA1.3.2147428157.1549110685,9223133706373664157-1611759332,BZJG1G8CY,3,2021-01-27,2021-01-27 14:55:32,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/error-page/500,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,NaN
1215045,-9223372036854775808,GA1.3.40df6747-c6e4-47a9-970a-bf5d8890634a,13681945308945359288-1617108480,NaN,1,2021-03-30,2021-03-30 12:48:00,Paid Search,google,cpc,...,0,0,www.fwd.com.sg.googleweblight.com/stroke-insur...,www.fwd.com.sg.googleweblight.com/stroke-insur...,NaN,NaN,NaN,NaN,NaN,NaN


## Mapping Page Title to Products

### Entrance Page Mapping

In [5]:
print("Initial Dataset:", len(df_session_small))

#Clearing low count pages
page_count = df_session_small['entrance_page_title'].value_counts()
remove_row = page_count[page_count <= 300].index
df_session_small = df_session_small[~df_session_small['entrance_page_title'].isin(remove_row)]

#Format Titles
df_session_small['entrance_page_title'] = df_session_small['entrance_page_title'].str.replace("\s: | FWD Singapore", "")
df_session_small['exit_page_title'] = df_session_small['exit_page_title'].str.replace("\s: | FWD Singapore", "")

# Get Data to be map by page path 
df_page_path = df_session_small.loc[((df_session_small['entrance_page_title'] == 'Insurance Promotions |') |
                                     (df_session_small['entrance_page_title'] == 'Insurance FAQs |') |
                                     (df_session_small['entrance_page_title'].isnull())
                                    )]

# Dropping Rows 
removal_list = ['Online Insurance Company, Insurance Claims |','FWD Online Login Portal |', 'FWD',
                'Insurance Promotions |', "Contact Us - We're Here to Help |", 'Maintenance Page', 
                'Insurance FAQs |', 'Find your nearest FWD workshop |', 'About Us |',
                'Insurance Promotions: Refer a Friend | FWD Singapore', 'This page could not be found', 'Future First |',
                'Endowment Savings & Investments Insurance |', 'FAQ - All You Need to Know About Claims |',
                'Insurance Promotions: Refer a Friend |',
                'Insurance Forms |', 'Find Out More About Us |']

df_session_small = df_session_small[~df_session_small['entrance_page_title'].isin(removal_list)]
df_session_small = df_session_small[df_session_small['entrance_page_title'].notna()]

#Consolidating product types (Page Titles)
mapping_title = {'Buy Car Insurance Plan Online |':'Car', 'HDB Fire Insurance Plans Online |':'HDB Fire', 'Car Insurance Online Quote |':'Car', 
                   'Buy Maid Insurance Plan Online |':'Maid', 'Buy Motorcycle Insurance Plan Online |':'Motor', 'Buy Term Life Insurance Online |':'Term Life', 
                   'Buy Home Insurance Online |':'Home', 'Home Insurance Online Quote |':'Home', 'Buy Critical Illness Insurance Plans Online |':'Big 3 Critical Illness', 
                   'Personal Accident Insurance Plan Online |':'Personal Accident', 'Maid Insurance Online Quote |':'Maid', 'Buy Travel Insurance Plan Online |':'Travel', 
                   'HDB Fire Plan - Instant Quote Online |':'HDB Fire', 'Buy Cancer Insurance Online |':'Cancer', 'Travel Insurance Online Quotes |':'Travel',
                   'Get an Online Motorcycle Insurance Quote |':'Motor', 'Buy International Health Insurance Plan |':'International Health', 'Buy Big 3 Critical Illness Insurance Plan Online |':'Big 3 Critical Illness',  
                   'Commercial Car Insurance |':'Commercial Car', 'Get an Online Big 3 Critical Illness Quote |':'Big 3 Critical Illness', 'Personal Accident Insurance Online Quote |':'Personal Accident', 
                   'Covid-19 insurance Online Quote |':'Covid-19', 'Buy Commercial Car Insurance Online |':'Commercial Car', 'Buy Covid-19 insurance Online |':'Covid-19', 
                   'Commercial Car Insurance - Comprehensive Motor Quote |':'Commercial Car', 'Get an Online Cancer Quote |':'Cancer', 'International Health Insurance Quote |':'International Health', 
                   'Get a Covid 19 Online Quote |':'Covid-19', 'Personal Insurance Claims & FAQs |':'Personal Accident', 'Buy Stroke Insurance Online |':'Stroke', 
                   'How Does Your NCD Work? |':'Personal Accident', 'Buy Cancer Insurance Plan Online |':'Cancer', 'Buy Heart Attack Insurance Online |':'Heart Attack', 
                   '4 Common Mistakes by Motorcyclists in Singapore |':'Motor', '3 Safe Driving Tips to Protect You & Your Loved Ones |':'Car', 'Car Insurance Claims Guide |':'Car', 
                   '5 Surprising Things Covered by Personal Accident insurance |':'Personal Accident', 'Direct Purchase Term Life Insurance Plans |':'Term Life', 'How to Start Off On The Right Foot With Your Maid |':'Maid', 
                   'Maid Insurance Claims Guide |':'Maid', 'Modular Maid Insurance Online Quote |':'Maid', 'Landlord Insurance vs. Homeowners Insurance |':'Home', 
                   'How living with a giant tumour changed my view on life |':'Cancer', 'Adventure Guide to Use SingapoRediscovers Voucher |':'Personal Accident', '5 Reasons You Need Home Insurance |':'Home', 
                   '3 Steps to Renewing Your Road Taxes |':'Car', '5 Parent-Child Bonding Ideas For School Holidays |':'Personal Accident', 'How to Save on Car Expenses in Singapore |':'Car', 
                   'How to Choose the Right Critical Illness Plan |':'Big 3 Critical Illness', 'Direct Term Life Online Quotes |':'Term Life', 'Get an Online Heart Attack Quote |' : 'Heart Attack',
                   'Get an Online Stroke Insurance Quote |': 'Stroke'
                  }

df_session_small['entrance_page_title'].replace(mapping_title, inplace=True)

print("Dataset after mapping:", len(df_session_small))


#Consolidating product types (Page Path)
page_count2 = df_page_path['entrance_page_path'].value_counts()
remove_row2 = page_count2[page_count2 <= 300].index
df_page_path = df_page_path[~df_page_path['entrance_page_path'].isin(remove_row2)]

mapping_path = {'www.fwd.com.sg/':'Homepage', 'www.fwd.com.sg/insurance-promotions/':'Promo', 'www.fwd.com.sg/error-page/404':'Error',
                'www.fwd.com.sg/error-page/500':'Error', 'www.fwd.com.sg/life-insurance/term-life-plus/online-quote/quickQuestions':'Term Life', 'www.fwd.com.sg/app/SGApp':'App',
                'www.fwd.com.sg/personalised-financial-advice/invest-first' : 'Invest First', 'www.fwd.com.sg/insurance-help-faqs/' : 'FAQ', 'www.fwd.com.sg/personalised-financial-advice/recover-first' : 'Recover First', 
                'www.fwd.com.sg/our-services/':'Services', 'www.fwd.com.sg/?promo=FWDDAYCS': 'Promo', 'www.fwd.com.sg/personalised-financial-advice/funds/' : 'Invest First',
                'www.fwd.com.sg/motorcycle-insurance/online-quote/renewPolicy':'Motor', 'www.fwd.com.sg/life-insurance/term-life-plus/online-quote/underWrite':'Term Life', 'www.fwd.com.sg/insurance-claims/ride-cover':'Personal Accident',
                'www.fwd.com.sg/#call':'Services', 'www.fwd.com.sg/life-insurance/term-life-plus/online-quote':'Term Life', 'www.fwd.com.sg/corporate/privacy-policy.html':'Privacy', 'www.fwd.com.sg/life-insurance/term-life-plus/online-quote/Application' : 'Term Life',
                'www.fwd.com.sg/personal-accident-renewal-insurance/online-quote/renewPolicy':'Personal Accident', 'www.fwd.com.sg/content/landingRedirect?code=a439c3f7-5d81-35b3-80d6-167cbce05dc1' : 'Term Life', 'Get an Online Heart Attack Quote |' : 'Heart Attack',
                'Get an Online Stroke Insurance Quote |': 'Stroke'
               }

df_page_path['entrance_page_path'].replace(mapping_path, inplace=True)

df_page_path = df_page_path[(df_page_path['entrance_page_path'] != "Homepage") & (df_page_path['entrance_page_path'] != "Error")
                            & (df_page_path['entrance_page_path'] != "App") & (df_page_path['entrance_page_path'] != "FAQ")
                            & (df_page_path['entrance_page_path'] != "Services") & (df_page_path['entrance_page_path'] != "Promo")
                            & (df_page_path['entrance_page_path'] != 'Privacy')]


# Adding mapped page path into df session
df_session_small = df_session_small.rename(columns = {'entrance_page_title':'Entry Product'})
df_page_path = df_page_path.rename(columns = {'entrance_page_path':'Entry Product'})

del df_session_small['entrance_page_path']
del df_page_path['entrance_page_title']

df_session_small = df_session_small.append(df_page_path)

print("Dataset after mapping path:", len(df_session_small))

Initial Dataset: 1215047


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Dataset after mapping: 850726
Dataset after mapping path: 852273


In [6]:
print("Inital Dataset:", len(df_session_small))

#Consolidating product types (Page Path)
page_count3 = df_session_small['exit_page_title'].value_counts()
remove_row3 = page_count3[page_count3 <= 300].index
df_session_small = df_session_small[~df_session_small['exit_page_title'].isin(remove_row3)]

# Get Data to be map by page path 
df_exitpage_path = df_session_small.loc[((df_session_small['exit_page_title'] == 'Insurance Promotions |') |
                                     (df_session_small['exit_page_title'] == 'Insurance FAQs |') |
                                     (df_session_small['exit_page_title'].isnull())
                                    )]

df_session_small = df_session_small[~df_session_small['exit_page_title'].isin(removal_list)]
df_session_small = df_session_small[df_session_small['exit_page_title'].notna()]

df_session_small['exit_page_title'].replace(mapping_title, inplace=True)

print("Dataset after mapping:", len(df_session_small))

df_session_small = df_session_small.rename(columns = {'exit_page_title':'Exit Product'})

del df_session_small['exit_page_path']

print("After deleting Dataset:", len(df_session_small))

Inital Dataset: 852273
Dataset after mapping: 775322
After deleting Dataset: 775322


In [7]:
# Dropping Columns
df_session_small = df_session_small.drop(columns=['client_id', 'session_date', 'mobiledevicebranding', 'country', 'session_id.1'])
df_session_small

,full_visitor_id,session_id,quotation_id,visit_number,session_date_time,channelgrouping,source,medium,keyword,adcontent,...,total_tl_pageviews,total_ih_pageviews,total_pa_pageviews,applicant_drop_out_flag,Entry Product,Exit Product,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
0,701696772471,701696772471-1617103223,NaN,1,2021-03-30 11:20:23,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
3,701696772471,701696772471-1617183736,NaN,4,2021-03-31 09:42:16,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
4,701696772471,701696772471-1617183736,8O3Z1JIIZ,4,2021-03-31 09:42:16,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,Maid,2021-03-30 17:21:03.247,0.0,1.0
5,701696772471,701696772471-1617183736,CETP1PS8B,4,2021-03-31 09:42:16,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
8,39674238326953,39674238326953-1625413801,CGM92326315QKFP,1,2021-07-04 15:50:01,(Other),Renewal,EDM,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,Maid,2019-07-07 16:01:52.377,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199134,9103898263197699072,9103898263197699238-1630734716,NaN,4,2021-09-04 05:51:56,Referral,link.sg.fwd.com,referral,(not set),(not set),...,0,0,0,0,Motor,Motor,NaN,NaN,NaN,NaN
1199135,9103898263197699072,9103898263197699238-1630734716,NaN,4,2021-09-04 05:51:56,Referral,link.sg.fwd.com,referral,(not set),(not set),...,0,0,0,0,Motor,Motor,NaN,NaN,NaN,NaN
1206290,9158506234023037952,9158506234023038202-1613024511,NaN,2,2021-02-11 06:21:51,Referral,link.sg.fwd.com,referral,(not set),(not set),...,0,0,0,0,Motor,Motor,NaN,NaN,NaN,NaN
1209721,9183137639556531200,9183137639556532135-1632192561,NaN,4,2021-09-21 02:49:21,Referral,link.sg.fwd.com,referral,(not set),(not set),...,0,0,0,0,Motor,Motor,NaN,NaN,NaN,NaN


In [9]:
basicDataExploration(df_session_small)

General Stats::
<class 'pandas.core.frame.DataFrame'>
Int64Index: 775322 entries, 0 to 1209722
Data columns (total 34 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   full_visitor_id              775322 non-null  int64  
 1   client_id                    775322 non-null  object 
 2   session_id                   775322 non-null  object 
 3   quotation_id                 272975 non-null  object 
 4   visit_number                 775322 non-null  int64  
 5   session_date                 775322 non-null  object 
 6   session_date_time            775322 non-null  object 
 7   channelgrouping              775322 non-null  object 
 8   source                       775322 non-null  object 
 9   medium                       775322 non-null  object 
 10  keyword                      775322 non-null  object 
 11  adcontent                    775322 non-null  object 
 12  referralpath                 775322 non-n

In [7]:
df_session_small.sort_values(['full_visitor_id','visit_number'])

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_tl_pageviews,total_ih_pageviews,total_pa_pageviews,applicant_drop_out_flag,Entry Product,Exit Product,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
0,701696772471,GA1.3.163.1617103223,701696772471-1617103223,NaN,1,2021-03-30,2021-03-30 11:20:23,(Other),Renewal,SMS,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
3,701696772471,GA1.3.163.1617103223,701696772471-1617183736,NaN,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
4,701696772471,GA1.3.163.1617103223,701696772471-1617183736,8O3Z1JIIZ,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,0,0,Maid,Maid,Maid,2021-03-30 17:21:03.247,0.0,1.0
5,701696772471,GA1.3.163.1617103223,701696772471-1617183736,CETP1PS8B,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
8,39674238326953,GA1.3.9237.1625413801,39674238326953-1625413801,CGM92326315QKFP,1,2021-07-04,2021-07-04 15:50:01,(Other),Renewal,EDM,...,0,0,0,0,Maid,Maid,Maid,2019-07-07 16:01:52.377,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215040,9223322530391868416,GA1.3.2147472121.1625114202,9223322530391869018-1625466530,NaN,3,2021-07-05,2021-07-05 06:28:50,Organic Search,google,organic,...,0,0,1,0,Personal Accident,Personal Accident,NaN,NaN,NaN,NaN
1215041,9223329934920336384,GA1.3.2147473845.1629963161,9223329934920336281-1629963160,8EUM1IQMZ,1,2021-08-26,2021-08-26 07:32:40,(Other),Renewal,SMS,...,0,0,0,0,Home,Home,NaN,NaN,NaN,NaN
1215042,9223329934920336384,GA1.3.2147473845.1629963161,9223329934920336281-1629963160,NaN,1,2021-08-26,2021-08-26 07:32:40,(Other),Renewal,SMS,...,0,0,0,0,Home,Home,NaN,NaN,NaN,NaN
1215043,9223352981715395584,GA1.3.2147479211.1630512433,9223352981715395889-1630512431,NaN,1,2021-09-02,2021-09-01 16:07:11,Paid Search,google,cpc,...,0,0,0,0,Car,Car,NaN,NaN,NaN,NaN


In [12]:
test = df_session_small.loc[df_session_small['full_visitor_id'] == 8668403250992289792]
test

,full_visitor_id,session_id,quotation_id,visit_number,session_date_time,channelgrouping,source,medium,keyword,adcontent,...,total_tl_pageviews,total_ih_pageviews,total_pa_pageviews,applicant_drop_out_flag,Entry Product,Exit Product,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
1141525,8668403250992289792,8668403250992289792-1631361024,NaN,1,2021-09-11 11:50:24,(Other),Renewal,EDM,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
1141526,8668403250992289792,8668403250992289792-1632146451,NaN,2,2021-09-20 14:00:51,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
1141527,8668403250992289792,8668403250992289792-1632146451,R53Z19BCK,2,2021-09-20 14:00:51,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
1141528,8668403250992289792,8668403250992289792-1633272754,NaN,3,2021-10-03 14:52:34,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
1141529,8668403250992289792,8668403250992289792-1633272754,R53Z19BCK,3,2021-10-03 14:52:34,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
1141530,8668403250992289792,8668403250992289792-1633664139,PRDA1782H,4,2021-10-08 03:35:39,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
1141531,8668403250992289792,8668403250992289792-1633664139,IVWI1ACDW,4,2021-10-08 03:35:39,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,Maid,2021-09-11 19:51:16.90,0.0,1.0
1141532,8668403250992289792,8668403250992289792-1633664139,R53Z19BCK,4,2021-10-08 03:35:39,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN
1141533,8668403250992289792,8668403250992289792-1633664139,NaN,4,2021-10-08 03:35:39,(Other),Renewal,SMS,(not set),DiscountYes,...,0,0,0,0,Maid,Maid,NaN,NaN,NaN,NaN


In [14]:
df_session_small['full_visitor_id'].value_counts()

8668403250992289792    9
2531636468156338176    9
4822142767620553728    9
1069311738657461632    9
794409805665829632     9
                      ..
5478746570293303296    1
2327629928240668160    1
1623465453034092288    1
8982785077412782080    1
8193762416491589632    1
Name: full_visitor_id, Length: 286263, dtype: int64

In [19]:
my_value_counts(df_session_small, 'exit_page_title')

,Frequency,Proportion
Car,200162,0.258200
HDB Fire,91634,0.118200
Maid,90637,0.116900
Home,85502,0.110300
Motor,72862,0.094000
Big 3 Critical Illness,48900,0.063100
Personal Accident,38717,0.050000
Travel,38358,0.049500
Term Life,38275,0.049400
Cancer,23771,0.030700


In [17]:
#Consolidating product types (Page Path)
page_count4 = df_exitpage_path['exit_page_path'].value_counts()
remove_row4 = page_count4[page_count4 <= 300].index
df_exitpage_path = df_exitpage_path[~df_exitpage_path['exit_page_path'].isin(remove_row4)]

In [58]:
my_value_counts(df_session_small, 'entrance_page_title')

,Frequency,Proportion
Car,222116,0.261100
HDB Fire,101625,0.119500
Maid,97007,0.114000
Home,93311,0.109700
Motor,75165,0.088400
Term Life,54885,0.064500
Big 3 Critical Illness,51448,0.060500
Travel,42212,0.049600
Personal Accident,39275,0.046200
Cancer,25991,0.030600


In [56]:
my_value_counts(df_page_path, 'entrance_page_path')

,Frequency,Proportion
Term Life,5858,0.497300
Invest First,3080,0.261500
Recover First,1150,0.097600
Motor,896,0.076100
Personal Accident,796,0.067600
Total,11780,1.000000


In [84]:
my_value_counts(df_session_small, 'Entry Product')

,Frequency,Proportion
Car,222116,0.261100
HDB Fire,101625,0.119500
Maid,97007,0.114000
Home,93311,0.109700
Motor,75165,0.088400
Term Life,54885,0.064500
Big 3 Critical Illness,51448,0.060500
Travel,42212,0.049600
Personal Accident,39275,0.046200
Cancer,25991,0.030600


In [116]:
my_value_counts(df_session_small, 'Entry Product')

,Frequency,Proportion
Car,222116,0.260200
HDB Fire,101625,0.119000
Maid,97007,0.113600
Home,93311,0.109300
Motor,76061,0.089100
Term Life,55210,0.064700
Big 3 Critical Illness,51448,0.060300
Travel,42212,0.049400
Personal Accident,39601,0.046400
Cancer,25991,0.030400


In [89]:
df_session_small

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_ih_pageviews,total_pa_pageviews,applicant_drop_out_flag,exit_page_path,Entry Product,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
0,701696772471,GA1.3.163.1617103223,701696772471-1617103223,NaN,1,2021-03-30,2021-03-30 11:20:23,(Other),Renewal,SMS,...,0,0,0,www.fwd.com.sg/maid-insurance/online-quote/ren...,Maid,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
1,701696772471,GA1.3.163.1617103223,701696772471-1617183010,NaN,2,2021-03-31,2021-03-31 09:30:10,(Other),Renewal,SMS,...,0,0,0,www.fwd.com.sg/portal/,Maid,FWD Online Login Portal | FWD Singapore,NaN,NaN,NaN,NaN
3,701696772471,GA1.3.163.1617103223,701696772471-1617183736,NaN,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,0,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
4,701696772471,GA1.3.163.1617103223,701696772471-1617183736,8O3Z1JIIZ,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,0,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid,Maid Insurance Online Quote | FWD Singapore,Maid,2021-03-30 17:21:03.247,0.0,1.0
5,701696772471,GA1.3.163.1617103223,701696772471-1617183736,CETP1PS8B,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,0,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214793,9221725614293836800,GA1.3.2147100310.1612374866,9221725614293836626-1612666318,NaN,6,2021-02-07,2021-02-07 02:51:58,Organic Search,google,organic,...,0,0,0,www.fwd.com.sg/life-insurance/term-life-plus/o...,Term Life,"Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
1214795,9221725614293836800,GA1.3.2147100310.1612374866,9221725614293836626-1612764468,NaN,8,2021-02-08,2021-02-08 06:07:48,Organic Search,google,organic,...,0,0,0,www.fwd.com.sg/life-insurance/term-life-plus/o...,Term Life,"Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
1214796,9221725614293836800,GA1.3.2147100310.1612374866,9221725614293836626-1612882001,NaN,9,2021-02-09,2021-02-09 14:46:41,Organic Search,google,organic,...,0,0,0,www.fwd.com.sg/life-insurance/term-life-plus/o...,Term Life,"Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
1214878,9222353040297495552,GA1.3.2147246394.1613563644,9222353040297494268-1613731136,NaN,4,2021-02-19,2021-02-19 10:38:56,Display,dbm,cpm,...,0,0,0,www.fwd.com.sg/life-insurance/term-life-plus/o...,Term Life,"Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN


In [97]:
df_session_small['exit_page_title'].replace(mapping_title, inplace=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [110]:
my_value_counts(df_session_small, 'exit_page_title')

,Frequency,Proportion
Car,200184,0.257700
HDB Fire,91642,0.118000
Maid,90646,0.116700
Home,85526,0.110100
Motor,72992,0.094000
Big 3 Critical Illness,49021,0.063100
Personal Accident,38790,0.049900
Term Life,38540,0.049600
Travel,38364,0.049400
Cancer,23822,0.030700


In [113]:
# Get Data to be map by page path 
df_exitpage_path = df_session_small.loc[((df_session_small['exit_page_title'] == 'Insurance Promotions |') |
                                     (df_session_small['exit_page_title'] == 'Insurance FAQs |') |
                                     (df_session_small['exit_page_title'].isnull())
                                    )]

df_exitpage_path

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
186,1012389843896307,GA1.3.235715.1627719667,1012389843896307-1628644703,NaN,3,2021-08-11,2021-08-11 01:18:23,(Other),Renewal,EDM,...,2,0,www.fwd.com.sg/personal-accident-insurance/ren...,www.fwd.com.sg/personal-accident-renewal-insur...,NaN,NaN,NaN,NaN,NaN,NaN
240,1218612696524046,GA1.3.283730.1625629966,1218612696524046-1626865323,NaN,3,2021-07-21,2021-07-21 11:02:03,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/error-page/404,www.fwd.com.sg/error-page/404,NaN,NaN,NaN,NaN,NaN,NaN
320,1899386485110222,GA1.3.442235.1622963662,1899386485110222-1622963661,NaN,1,2021-06-06,2021-06-06 07:14:21,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/car-insurance/renewal.html?id=1...,www.fwd.com.sg/app/SGApp,Car Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,NaN
328,1993961663615368,GA1.3.464255.1621610889,1993961663615369-1621655050,NaN,2,2021-05-22,2021-05-22 03:44:10,Direct,(direct),(none),...,0,0,www.fwd.com.sg/error-page/500,www.fwd.com.sg/error-page/500,NaN,NaN,NaN,NaN,NaN,NaN
378,2381745682206586,GA1.3.554543.1632980858,2381745682206586-1633417577,NaN,5,2021-10-05,2021-10-05 07:06:17,Direct,(direct),(none),...,0,0,www.fwd.com.sg/app/SGApp,www.fwd.com.sg/app/SGApp,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215004,9223027354476510208,GA1.3.2147403395.1632140691,9223027354476510611-1632876890,NaN,4,2021-09-29,2021-09-29 00:54:50,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/car-insurance/renewal.html?id=b...,www.fwd.com.sg/app/SGApp,Car Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,NaN
1215016,9223133706373663744,GA1.3.2147428157.1549110685,9223133706373664157-1611759332,NaN,3,2021-01-27,2021-01-27 14:55:32,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/error-page/500,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,NaN
1215017,9223133706373663744,GA1.3.2147428157.1549110685,9223133706373664157-1611759332,BZJG1G8CY,3,2021-01-27,2021-01-27 14:55:32,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/error-page/500,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,NaN
1215045,-9223372036854775808,GA1.3.40df6747-c6e4-47a9-970a-bf5d8890634a,13681945308945359288-1617108480,NaN,1,2021-03-30,2021-03-30 12:48:00,Paid Search,google,cpc,...,0,0,www.fwd.com.sg.googleweblight.com/stroke-insur...,www.fwd.com.sg.googleweblight.com/stroke-insur...,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
df_session_small = df_session_small[~df_session_small['exit_page_title'].isin(removal_list)]
df_session_small = df_session_small[df_session_small['exit_page_title'].notna()]

In [ ]:
# wORKINGS FOR METHODS

In [81]:
df_session_small = df_session_small.rename(columns = {'entrance_page_title':'Entry Product'})
df_page_path = df_page_path.rename(columns = {'entrance_page_path':'Entry Product'})

del df_session_small['entrance_page_path']
del df_page_path['entrance_page_title']

df_session_small = df_session_small.append(df_page_path)

In [51]:
page_count2 = df_page_path['entrance_page_path'].value_counts()
remove_row2 = page_count2[page_count2 <= 300].index
df_page_path = df_page_path[~df_page_path['entrance_page_path'].isin(remove_row2)]

In [53]:
mapping_path = {'www.fwd.com.sg/':'Homepage', 'www.fwd.com.sg/insurance-promotions/':'Promo', 'www.fwd.com.sg/error-page/404':'Error',
                'www.fwd.com.sg/error-page/500':'Error', 'www.fwd.com.sg/life-insurance/term-life-plus/online-quote/quickQuestions':'Term Life', 'www.fwd.com.sg/app/SGApp':'App',
                'www.fwd.com.sg/personalised-financial-advice/invest-first' : 'Invest First', 'www.fwd.com.sg/insurance-help-faqs/' : 'FAQ', 'www.fwd.com.sg/personalised-financial-advice/recover-first' : 'Recover First', 
                'www.fwd.com.sg/our-services/':'Services', 'www.fwd.com.sg/?promo=FWDDAYCS': 'Promo', 'www.fwd.com.sg/personalised-financial-advice/funds/' : 'Invest First',
                'www.fwd.com.sg/motorcycle-insurance/online-quote/renewPolicy':'Motor', 'www.fwd.com.sg/life-insurance/term-life-plus/online-quote/underWrite':'Term Life', 'www.fwd.com.sg/insurance-claims/ride-cover':'Personal Accident',
                'www.fwd.com.sg/#call':'Services', 'www.fwd.com.sg/life-insurance/term-life-plus/online-quote':'Term Life', 'www.fwd.com.sg/corporate/privacy-policy.html':'Privacy', 'www.fwd.com.sg/life-insurance/term-life-plus/online-quote/Application' : 'Term Life',
                'www.fwd.com.sg/personal-accident-renewal-insurance/online-quote/renewPolicy':'Personal Accident', 'www.fwd.com.sg/content/landingRedirect?code=a439c3f7-5d81-35b3-80d6-167cbce05dc1' : 'Term Life'
               }

df_page_path['entrance_page_path'].replace(mapping_path, inplace=True)


In [55]:
df_page_path = df_page_path[(df_page_path['entrance_page_path'] != "Homepage") & (df_page_path['entrance_page_path'] != "Error")
                            & (df_page_path['entrance_page_path'] != "App") & (df_page_path['entrance_page_path'] != "FAQ")
                            & (df_page_path['entrance_page_path'] != "Services") & (df_page_path['entrance_page_path'] != "Promo")
                            & (df_page_path['entrance_page_path'] != 'Privacy')]

In [42]:
path_index = df_page_path[(df_page_path['entrance_page_path'] == 'Homepage') & (df_page_path['entrance_page_path'] == 'Error') & (df_page_path['entrance_page_path'] == 'App') &
                          (df_page_path['entrance_page_path'] == 'FAQ') & (df_page_path['entrance_page_path'] == 'Services') & (df_page_path['entrance_page_path'] == 'Promo') &
                          (df_page_path['entrance_page_path'] == 'Privacy')
                         ].index


df_page_path.drop(path_index, inplace=True)

In [18]:
df_session_small['entrance_page_title'] = df_session_small['entrance_page_title'].str.replace("\s: | FWD Singapore", "")
df_session_small['entrance_page_title']

0                      Maid Insurance Online Quote |
1                      Maid Insurance Online Quote |
3                      Maid Insurance Online Quote |
4                      Maid Insurance Online Quote |
5                      Maid Insurance Online Quote |
                             ...                    
1215040    Personal Accident Insurance Plan Online |
1215041                Home Insurance Online Quote |
1215042                Home Insurance Online Quote |
1215043              Buy Car Insurance Plan Online |
1215044              Buy Car Insurance Plan Online |
Name: entrance_page_title, Length: 850726, dtype: object

In [20]:
#Consolidating product types
mapping_product = {'Buy Car Insurance Plan Online |':'Car', 'HDB Fire Insurance Plans Online |':'HDB Fire', 'Car Insurance Online Quote |':'Car', 
                   'Buy Maid Insurance Plan Online |':'Maid', 'Buy Motorcycle Insurance Plan Online |':'Motor', 'Buy Term Life Insurance Online |':'Term Life', 
                   'Buy Home Insurance Online |':'Home', 'Home Insurance Online Quote |':'Home', 'Buy Critical Illness Insurance Plans Online |':'Big 3 Critical Illness', 
                  }

df_session_small['entrance_page_title'].replace(mapping_product, inplace=True)

In [43]:
df_session_small.loc[df_session_small['entrance_page_title'] == 'Insurance Promotions: Refer a Friend | FWD Singapore']['entrance_page_path'].value_counts()[:20]

www.fwd.com.sg/referrals                                                                                                   1091
www.fwd.com.sg/referrals/                                                                                                    60
www.fwd.com.sg/referrals?sc_src=email_1325469&sc_lid=81677271&sc_uid=esNpYoYOWZ&sc_llid=20816&sc_eh=1563895a709b37471         7
www.fwd.com.sg/referrals?sc_src=email_1715329&sc_lid=114281556&sc_uid=br7mBfDT43&sc_llid=137341&sc_eh=2d69d8fea93cd7fe1       4
www.fwd.com.sg/referrals?sc_src=email_1729981&sc_lid=115604556&sc_uid=eZAsy4BBvU&sc_llid=17907&sc_eh=d69e78a35474a82b1        4
www.fwd.com.sg/referrals?sc_src=email_1715329&sc_lid=114281556&sc_uid=PXzDEspaFp&sc_llid=89085&sc_eh=b38fd4d11de8ace11        4
www.fwd.com.sg/referrals?sc_src=email_1715329&sc_lid=114281556&sc_uid=dbh9Zw1yCl&sc_llid=38335&sc_eh=0407a8b7497df8041        4
www.fwd.com.sg/referrals?sc_src=email_1715329&sc_lid=114281556&sc_uid=HWGzgwX3fX&sc_llid=90402&sc_eh=c0f

In [38]:
df_session_small.loc[df_session_small['entrance_page_title'] == 'Insurance FAQs | FWD Singapore']['entrance_page_path'].value_counts()[:20]

www.fwd.com.sg/insurance-help-faqs/                                                                                                                                                                                                                          2056
www.fwd.com.sg/insurance-help-faqs/#cancer                                                                                                                                                                                                                    158
www.fwd.com.sg/insurance-help-faqs/#car                                                                                                                                                                                                                       117
www.fwd.com.sg/insurance-help-faqs/#home                                                                                                                                                                                          

In [57]:
df_session_small.loc[df_session_small['entrance_page_title'] == 'Online Insurance Company, Insurance Claims | FWD Singapore']

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
2,701696772471,GA1.3.163.1617103223,701696772471-1617183311,NaN,3,2021-03-31,2021-03-31 09:35:11,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
6,32596128723860,GA1.3.7589.1621914516,32596128723860-1621914518,NaN,1,2021-05-25,2021-05-25 03:48:38,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/insurance-promotions/,"Online Insurance Company, Insurance Claims | F...",Insurance Promotions | FWD Singapore,NaN,NaN,NaN,NaN
7,32596128723860,GA1.3.7589.1621914516,32596128723860-1622689656,NaN,2,2021-06-03,2021-06-03 03:07:36,Organic Search,google,organic,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/portal/,"Online Insurance Company, Insurance Claims | F...",FWD Online Login Portal | FWD Singapore,NaN,NaN,NaN,NaN
19,97179542174497,GA1.3.22626.1612135201,97179542174497-1612353961,NaN,2,2021-02-03,2021-02-03 12:06:01,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
20,97179542174497,GA1.3.22626.1612135201,97179542174497-1612356745,NaN,3,2021-02-03,2021-02-03 12:52:25,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214958,9222798741955120128,GA1.3.2147350167.1629981397,9222798741955119829-1629981396,NaN,1,2021-08-26,2021-08-26 12:36:36,Direct,(direct),(none),...,0,0,www.fwd.com.sg/,www.fwd.com.sg/portal/password.html,"Online Insurance Company, Insurance Claims | F...",FWD,NaN,NaN,NaN,NaN
1214963,9222799205793752064,GA1.3.2147350275.1612144416,9222799205793750816-1612144416,NaN,1,2021-02-01,2021-02-01 01:53:36,Direct,(direct),(none),...,0,0,www.fwd.com.sg/life-insurance/term-life-plus/o...,www.fwd.com.sg/portal/home.html,"Online Insurance Company, Insurance Claims | F...",FWD,NaN,NaN,NaN,NaN
1214964,9222799205793752064,GA1.3.2147350275.1612144416,9222799205793750816-1612144416,6GQV4418446TTKK,1,2021-02-01,2021-02-01 01:53:36,Direct,(direct),(none),...,0,0,www.fwd.com.sg/life-insurance/term-life-plus/o...,www.fwd.com.sg/portal/home.html,"Online Insurance Company, Insurance Claims | F...",FWD,Term Life Plus,2021-01-29 21:04:44.69,0.0,1.0
1214998,9223020847597437952,GA1.3.2147401880.1628521407,9223020847597437887-1630901708,UXFV4953689UARM,3,2021-09-06,2021-09-06 04:15:08,(Other),CROSSSELL,EDM,...,0,0,www.fwd.com.sg/?promo=FWDDAYCS&sc_src=email_18...,www.fwd.com.sg/maid-insurance/online-quote/tha...,"Online Insurance Company, Insurance Claims | F...",Maid Insurance Online Quote | FWD Singapore,Maid,2021-09-06 12:17:02.277,0.0,1.0


In [58]:
# Get Data to be map by page path 
df_page_path = df_session_small.loc[((df_session_small['entrance_page_title'] == 'Online Insurance Company, Insurance Claims | FWD Singapore') | 
                                     (df_session_small['entrance_page_title'] == 'Insurance Promotions | FWD Singapore') |
                                     (df_session_small['entrance_page_title'] == 'Insurance FAQs | FWD Singapore') |
                                     (df_session_small['entrance_page_title'].isnull())
                                    )] 

df_page_path

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
2,701696772471,GA1.3.163.1617103223,701696772471-1617183311,NaN,3,2021-03-31,2021-03-31 09:35:11,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
6,32596128723860,GA1.3.7589.1621914516,32596128723860-1621914518,NaN,1,2021-05-25,2021-05-25 03:48:38,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/insurance-promotions/,"Online Insurance Company, Insurance Claims | F...",Insurance Promotions | FWD Singapore,NaN,NaN,NaN,NaN
7,32596128723860,GA1.3.7589.1621914516,32596128723860-1622689656,NaN,2,2021-06-03,2021-06-03 03:07:36,Organic Search,google,organic,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/portal/,"Online Insurance Company, Insurance Claims | F...",FWD Online Login Portal | FWD Singapore,NaN,NaN,NaN,NaN
19,97179542174497,GA1.3.22626.1612135201,97179542174497-1612353961,NaN,2,2021-02-03,2021-02-03 12:06:01,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
20,97179542174497,GA1.3.22626.1612135201,97179542174497-1612356745,NaN,3,2021-02-03,2021-02-03 12:52:25,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214995,9223017888362882048,GA1.3.2147401191.1626432640,9223017888362882176-1626658590,NaN,3,2021-07-19,2021-07-19 01:36:30,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/error-page/500,www.fwd.com.sg/error-page/500,NaN,NaN,NaN,NaN,NaN,NaN
1214998,9223020847597437952,GA1.3.2147401880.1628521407,9223020847597437887-1630901708,UXFV4953689UARM,3,2021-09-06,2021-09-06 04:15:08,(Other),CROSSSELL,EDM,...,0,0,www.fwd.com.sg/?promo=FWDDAYCS&sc_src=email_18...,www.fwd.com.sg/maid-insurance/online-quote/tha...,"Online Insurance Company, Insurance Claims | F...",Maid Insurance Online Quote | FWD Singapore,Maid,2021-09-06 12:17:02.277,0.0,1.0
1214999,9223020847597437952,GA1.3.2147401880.1628521407,9223020847597437887-1630901708,NaN,3,2021-09-06,2021-09-06 04:15:08,(Other),CROSSSELL,EDM,...,0,0,www.fwd.com.sg/?promo=FWDDAYCS&sc_src=email_18...,www.fwd.com.sg/maid-insurance/online-quote/tha...,"Online Insurance Company, Insurance Claims | F...",Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
1215045,-9223372036854775808,GA1.3.40df6747-c6e4-47a9-970a-bf5d8890634a,13681945308945359288-1617108480,NaN,1,2021-03-30,2021-03-30 12:48:00,Paid Search,google,cpc,...,0,0,www.fwd.com.sg.googleweblight.com/stroke-insur...,www.fwd.com.sg.googleweblight.com/stroke-insur...,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_session_small['Entry Product'].value_counts()

Maid    855278
Name: Entry Product, dtype: int64

In [11]:
df_session_small

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator,Entry Product
0,701696772471,GA1.3.163.1617103223,701696772471-1617103223,NaN,1,2021-03-30,2021-03-30 11:20:23,(Other),Renewal,SMS,...,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/ren...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,International Health
1,701696772471,GA1.3.163.1617103223,701696772471-1617183010,NaN,2,2021-03-31,2021-03-31 09:30:10,(Other),Renewal,SMS,...,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/portal/,Maid Insurance Online Quote | FWD Singapore,FWD Online Login Portal | FWD Singapore,NaN,NaN,NaN,NaN,International Health
3,701696772471,GA1.3.163.1617103223,701696772471-1617183736,NaN,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,International Health
4,701696772471,GA1.3.163.1617103223,701696772471-1617183736,8O3Z1JIIZ,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,Maid,2021-03-30 17:21:03.247,0.0,1.0,International Health
5,701696772471,GA1.3.163.1617103223,701696772471-1617183736,CETP1PS8B,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,International Health
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215040,9223322530391868416,GA1.3.2147472121.1625114202,9223322530391869018-1625466530,NaN,3,2021-07-05,2021-07-05 06:28:50,Organic Search,google,organic,...,0,www.fwd.com.sg/personal-accident-insurance/,www.fwd.com.sg/personal-accident-insurance/,Personal Accident Insurance Plan Online | FWD ...,Personal Accident Insurance Plan Online | FWD ...,NaN,NaN,NaN,NaN,International Health
1215041,9223329934920336384,GA1.3.2147473845.1629963161,9223329934920336281-1629963160,8EUM1IQMZ,1,2021-08-26,2021-08-26 07:32:40,(Other),Renewal,SMS,...,0,www.fwd.com.sg/home-insurance/renewal.html?id=...,www.fwd.com.sg/home-insurance/online-quote/dem...,Home Insurance Online Quote | FWD Singapore,Home Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,International Health
1215042,9223329934920336384,GA1.3.2147473845.1629963161,9223329934920336281-1629963160,NaN,1,2021-08-26,2021-08-26 07:32:40,(Other),Renewal,SMS,...,0,www.fwd.com.sg/home-insurance/renewal.html?id=...,www.fwd.com.sg/home-insurance/online-quote/dem...,Home Insurance Online Quote | FWD Singapore,Home Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,International Health
1215043,9223352981715395584,GA1.3.2147479211.1630512433,9223352981715395889-1630512431,NaN,1,2021-09-02,2021-09-01 16:07:11,Paid Search,google,cpc,...,0,www.fwd.com.sg/car-insurance/,www.fwd.com.sg/car-insurance/comprehensive-mot...,Buy Car Insurance Plan Online | FWD Singapore,Car Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN,International Health


In [25]:
for i in df_session_small['entrance_page_title']:
    if i.str.contains("Car", case=False, na=False):
        df_session_small['Entry Product'] = "Car"
    elif i.contains("HDB Fire", case=False, na=False):
        df_session_small['Entry Product'] = "HDB Fire"
    elif i.contains("Maid ", case=False, na=False):
        df_session_small['Entry Product'] = "Maid"
    elif i.contains("Motorcycle ", case=False, na=False):
        df_session_small['Entry Product'] = "Motorcycle "
    elif i.contains("Term Life", case=False, na=False):
        df_session_small['Entry Product'] = "Term Life"
    elif i.contains("Home ", case=False, na=False):
        df_session_small['Entry Product'] = "Home"
    elif i.contains("Critical Illness ", case=False, na=False):
        df_session_small['Entry Product'] = "Big 3"
    elif i.contains("Personal", case=False, na=False):
        df_session_small['Entry Product'] = "Personal Accident"
    elif i.contains("Travel", case=False, na=False):
        df_session_small['Entry Product'] = "Travel" 
    elif i.contains("Cancer", case=False, na=False):
        df_session_small['Entry Product'] = "Cancer"
    elif i.contains("International Health", case=False, na=False):
        df_session_small['Entry Product'] = "International Health"
    elif i.contains("Commercial", case=False, na=False):
        df_session_small['Entry Product'] = "Commercial"
    elif i.contains("Covid-19", case=False, na=False):
        df_session_small['Entry Product'] = "Covid-19"
    elif i.contains("Future First", case=False, na=False):
        df_session_small['Entry Product'] = "Term Life"
    elif i.contains("Stroke", case=False, na=False):
        df_session_small['Entry Product'] = "Stroke"
    elif i.contains("Heart Attack", case=False, na=False):
        df_session_small['Entry Product'] = "Heart Attack"
    elif i.contains("NCD", case=False, na=False):
        df_session_small['Entry Product'] = "Car"
    elif i.contains("Driving ", case=False, na=False):
        df_session_small['Entry Product'] = "Car"
    elif i.contains("Homeowners", case=False, na=False):
        df_session_small['Entry Product'] = "Home"
    elif i.contains("tumour", case=False, na=False):
        df_session_small['Entry Product'] = "Cancer" 
    elif i.contains("SingapoRediscovers", case=False, na=False):
        df_session_small['Entry Product'] = "Personal Accident"   
    elif i.contains("Road", case=False, na=False):
        df_session_small['Entry Product'] = "Car"
    elif i.contains("Parent", case=False, na=False):
        df_session_small['Entry Product'] = "Personal Accident"

AttributeError: 'str' object has no attribute 'str'

In [9]:
# Dropping Rows 
removal_list = ['Online Insurance Company, Insurance Claims | FWD Singapore','FWD Online Login Portal | FWD Singapore', 'FWD',
                'Insurance Promotions | FWD Singapore', "Contact Us - We're Here to Help | FWD Singapore", 'Maintenance Page', 
                'Insurance FAQs | FWD Singapore', 'Find your nearest FWD workshop | FWD Singapore', 'About Us | FWD Singapore',
                'Insurance Promotions: Refer a Friend | FWD Singapore', 'This page could not be found', 
                'FAQ - All You Need to Know About Claims | FWD Singapore',
                'Insurance Forms | FWD Singapore', 'Find Out More About Us | FWD Singapore']

df_session_small = df_session_small[~df_session_small['entrance_page_title'].isin(removal_list)]
df_session_small = df_session_small[df_session_small['entrance_page_title'].notna()]

In [ ]:
(df_session_small['entrance_page_title'].isnull())

In [33]:
temp = df_session_small[df_session_small['entrance_page_title'].contains("HDB Fire", case=False, na=False)]
temp

AttributeError: 'Series' object has no attribute 'contains'

In [30]:

df_session_small["Entry Product"] = np.where(temp.str.contains("HDB Fire", case=False, na=False), "HDB Fire")
df_session_small["Entry Product"]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  app.launch_new_instance()


ValueError: either both or neither of x and y should be given

In [27]:
count = 0

for i in df_session_small['entrance_page_title']:
    if i.str.contains("HDB Fire", case=False, na=False):
        df_session_small["Entry Product"] = "HDB Fire"
    


AttributeError: 'str' object has no attribute 'str'

In [24]:
## Get rows with HDB Fire in page title
df_HDBfire = df_session_small.loc[df_session_small["entrance_page_title"].str.contains("HDB Fire", case=False, na=False)]
my_value_counts(df_HDBfire, 'entrance_page_title')

,Frequency,Proportion
HDB Fire Insurance Plans Online | FWD Singapore,78371,0.768600
HDB Fire Plan - Instant Quote Online | FWD Singapore,23254,0.228100
HDB Fire Insurance Claims Guide | FWD Singapore,181,0.001800
Home Insurance vs. HDB Fire Insurance: What is the Difference? | FWD Singapore,154,0.001500
Total,101960,1.000000


In [64]:
## Get rows with Maid in page title
df_Maid = df_session_small.loc[df_session_small["entrance_page_title"].str.contains("Maid", case=False, na=False)]
my_value_counts(df_Maid, 'entrance_page_title')

,Frequency,Proportion
Buy Maid Insurance Plan Online | FWD Singapore,67614,0.692100
Maid Insurance Online Quote | FWD Singapore,27978,0.286400
How to Start Off On The Right Foot With Your Maid | FWD Singapore,499,0.005100
Maid Insurance Claims Guide | FWD Singapore,461,0.004700
Modular Maid Insurance Online Quote | FWD Singapore,455,0.004700
3 Things Your Maid Insurance Should Cover | FWD Singapore,250,0.002600
4 Tips for Onboarding Your New Maid | FWD Singapore,192,0.002000
Learn About Maid Insurance Coverage | FWD Singapore,132,0.001400
FWD Revolutionises Maid Insurance | FWD Singapore,70,0.000700
Find Ways to Save When Hiring a Maid | FWD Singapore,24,0.000200


### Mapping Page Path to Products

In [57]:
### Analysing Claims Path
df_claims = df_session_small.loc[df_session_small['entrance_page_title'] == "Online Insurance Company, Insurance Claims | FWD Singapore"]
df_claims

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
2,701696772471,GA1.3.163.1617103223,701696772471-1617183311,NaN,3,2021-03-31,2021-03-31 09:35:11,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
6,32596128723860,GA1.3.7589.1621914516,32596128723860-1621914518,NaN,1,2021-05-25,2021-05-25 03:48:38,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/insurance-promotions/,"Online Insurance Company, Insurance Claims | F...",Insurance Promotions | FWD Singapore,NaN,NaN,NaN,NaN
7,32596128723860,GA1.3.7589.1621914516,32596128723860-1622689656,NaN,2,2021-06-03,2021-06-03 03:07:36,Organic Search,google,organic,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/portal/,"Online Insurance Company, Insurance Claims | F...",FWD Online Login Portal | FWD Singapore,NaN,NaN,NaN,NaN
19,97179542174497,GA1.3.22626.1612135201,97179542174497-1612353961,NaN,2,2021-02-03,2021-02-03 12:06:01,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
20,97179542174497,GA1.3.22626.1612135201,97179542174497-1612356745,NaN,3,2021-02-03,2021-02-03 12:52:25,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214958,9222798741955120128,GA1.3.2147350167.1629981397,9222798741955119829-1629981396,NaN,1,2021-08-26,2021-08-26 12:36:36,Direct,(direct),(none),...,0,0,www.fwd.com.sg/,www.fwd.com.sg/portal/password.html,"Online Insurance Company, Insurance Claims | F...",FWD,NaN,NaN,NaN,NaN
1214963,9222799205793752064,GA1.3.2147350275.1612144416,9222799205793750816-1612144416,NaN,1,2021-02-01,2021-02-01 01:53:36,Direct,(direct),(none),...,0,0,www.fwd.com.sg/life-insurance/term-life-plus/o...,www.fwd.com.sg/portal/home.html,"Online Insurance Company, Insurance Claims | F...",FWD,NaN,NaN,NaN,NaN
1214964,9222799205793752064,GA1.3.2147350275.1612144416,9222799205793750816-1612144416,6GQV4418446TTKK,1,2021-02-01,2021-02-01 01:53:36,Direct,(direct),(none),...,0,0,www.fwd.com.sg/life-insurance/term-life-plus/o...,www.fwd.com.sg/portal/home.html,"Online Insurance Company, Insurance Claims | F...",FWD,Term Life Plus,2021-01-29 21:04:44.69,0.0,1.0
1214998,9223020847597437952,GA1.3.2147401880.1628521407,9223020847597437887-1630901708,UXFV4953689UARM,3,2021-09-06,2021-09-06 04:15:08,(Other),CROSSSELL,EDM,...,0,0,www.fwd.com.sg/?promo=FWDDAYCS&sc_src=email_18...,www.fwd.com.sg/maid-insurance/online-quote/tha...,"Online Insurance Company, Insurance Claims | F...",Maid Insurance Online Quote | FWD Singapore,Maid,2021-09-06 12:17:02.277,0.0,1.0


In [17]:
df_session_null = df_session_small[df_session_small.filter(like='_page_title').isna().all(1)]
df_session_null

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
656875,1.012390e+15,GA1.3.235715.1627719667,1012389843896307-1628644703,NaN,3,2021-08-11,2021-08-11 01:18:23,(Other),Renewal,EDM,...,2,0,www.fwd.com.sg/personal-accident-insurance/ren...,www.fwd.com.sg/personal-accident-renewal-insur...,NaN,NaN,NaN,NaN,NaN,NaN
1169027,1.218613e+15,GA1.3.283730.1625629966,1218612696524046-1626865323,NaN,3,2021-07-21,2021-07-21 11:02:03,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/error-page/404,www.fwd.com.sg/error-page/404,NaN,NaN,NaN,NaN,NaN,NaN
731046,1.993962e+15,GA1.3.464255.1621610889,1993961663615369-1621655050,NaN,2,2021-05-22,2021-05-22 03:44:10,Direct,(direct),(none),...,0,0,www.fwd.com.sg/error-page/500,www.fwd.com.sg/error-page/500,NaN,NaN,NaN,NaN,NaN,NaN
414367,2.381746e+15,GA1.3.554543.1632980858,2381745682206586-1633417577,NaN,5,2021-10-05,2021-10-05 07:06:17,Direct,(direct),(none),...,0,0,www.fwd.com.sg/app/SGApp,www.fwd.com.sg/app/SGApp,NaN,NaN,NaN,NaN,NaN,NaN
101768,2.564836e+15,GA1.3.597172.1619509974,2564835829596886-1619532691,NaN,2,2021-04-27,2021-04-27 14:11:31,(Other),Renewal,EDM,...,0,0,www.fwd.com.sg/error-page/404,www.fwd.com.sg/error-page/404,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677459,9.222529e+18,GA1.3.2147287274.1615007140,9222528618561998244-1616667185,NaN,4,2021-03-25,2021-03-25 10:13:05,(Other),Renewal,EDM,...,15,0,www.fwd.com.sg/personal-accident-insurance/ren...,www.fwd.com.sg/personal-accident-renewal-insur...,NaN,NaN,NaN,NaN,NaN,NaN
1095714,9.223018e+18,GA1.3.2147401191.1626432640,9223017888362882176-1626449793,NaN,2,2021-07-16,2021-07-16 15:36:33,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/error-page/500,www.fwd.com.sg/error-page/500,NaN,NaN,NaN,NaN,NaN,NaN
716758,9.223018e+18,GA1.3.2147401191.1626432640,9223017888362882176-1626658590,NaN,3,2021-07-19,2021-07-19 01:36:30,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/error-page/500,www.fwd.com.sg/error-page/500,NaN,NaN,NaN,NaN,NaN,NaN
1085543,1.368195e+19,GA1.3.40df6747-c6e4-47a9-970a-bf5d8890634a,13681945308945359288-1617108480,NaN,1,2021-03-30,2021-03-30 12:48:00,Paid Search,google,cpc,...,0,0,www.fwd.com.sg.googleweblight.com/stroke-insur...,www.fwd.com.sg.googleweblight.com/stroke-insur...,NaN,NaN,NaN,NaN,NaN,NaN


### Extraction of Session Quality

### Target Variables

In [79]:
test = df_session_small.loc[df_session_small['full_visitor_id']  == 701696772471]
test

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
0,701696772471,GA1.3.163.1617103223,701696772471-1617103223,NaN,1,2021-03-30,2021-03-30 11:20:23,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/ren...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
1,701696772471,GA1.3.163.1617103223,701696772471-1617183010,NaN,2,2021-03-31,2021-03-31 09:30:10,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/portal/,Maid Insurance Online Quote | FWD Singapore,FWD Online Login Portal | FWD Singapore,NaN,NaN,NaN,NaN
2,701696772471,GA1.3.163.1617103223,701696772471-1617183311,NaN,3,2021-03-31,2021-03-31 09:35:11,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
3,701696772471,GA1.3.163.1617103223,701696772471-1617183736,NaN,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
4,701696772471,GA1.3.163.1617103223,701696772471-1617183736,8O3Z1JIIZ,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,Maid,2021-03-30 17:21:03.247,0.0,1.0
5,701696772471,GA1.3.163.1617103223,701696772471-1617183736,CETP1PS8B,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN


In [13]:
test_hits = df_home_small[df_home_small['fullvisitorid'] == 701696772471]
test_hits

,date,fullvisitorid,clientid,visitnumber,uniquevisitid,visitstarttime,visitdate,channelgrouping,source,medium,...,minute,isentrance,isexit,pagepath,pagepathlevel1,pagepathlevel2,pagepathlevel3,pagepathlevel4,pagetitle,type
122663,20210330,701696772471,GA1.3.163.1617103223,1,701696772471-1617103223,2021-03-30 11:20:23,2021-03-30,(Other),Renewal,SMS,...,20,NaN,t,www.fwd.com.sg/maid-insurance/online-quote/ren...,www.fwd.com.sg/,/maid-insurance/,/online-quote/,/renewPolicy,Maid Insurance Online Quote | FWD Singapore,PAGE
122664,20210330,701696772471,GA1.3.163.1617103223,1,701696772471-1617103223,2021-03-30 11:20:23,2021-03-30,(Other),Renewal,SMS,...,20,NaN,NaN,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/,/maid-insurance/,/renewal.html?id=560e29e5-83a2-4aa3-b7f5-a22ca...,NaN,Maid Insurance Online Quote | FWD Singapore,PAGE
122665,20210330,701696772471,GA1.3.163.1617103223,1,701696772471-1617103223,2021-03-30 11:20:23,2021-03-30,(Other),Renewal,SMS,...,20,t,NaN,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/,/maid-insurance/,/renewal.html?id=560e29e5-83a2-4aa3-b7f5-a22ca...,NaN,Maid Insurance Online Quote | FWD Singapore,PAGE
530773,20210331,701696772471,GA1.3.163.1617103223,4,701696772471-1617183736,2021-03-31 09:42:16,2021-03-31,(Other),Renewal,SMS,...,51,NaN,t,www.fwd.com.sg/maid-insurance/online-quote/dem...,www.fwd.com.sg/,/maid-insurance/,/online-quote/,/demographicquestions,Maid Insurance Online Quote | FWD Singapore,PAGE
530774,20210331,701696772471,GA1.3.163.1617103223,4,701696772471-1617183736,2021-03-31 09:42:16,2021-03-31,(Other),Renewal,SMS,...,51,NaN,NaN,www.fwd.com.sg/maid-insurance/online-quote/pay...,www.fwd.com.sg/,/maid-insurance/,/online-quote/,/payment-processing?type=renewal&policy=356436735,Maid Insurance Online Quote | FWD Singapore,PAGE
530775,20210331,701696772471,GA1.3.163.1617103223,4,701696772471-1617183736,2021-03-31 09:42:16,2021-03-31,(Other),Renewal,SMS,...,51,NaN,NaN,www.fwd.com.sg/maid-insurance/online-quote/dem...,www.fwd.com.sg/,/maid-insurance/,/online-quote/,/demographicquestions,Maid Insurance Online Quote | FWD Singapore,PAGE
530776,20210331,701696772471,GA1.3.163.1617103223,4,701696772471-1617183736,2021-03-31 09:42:16,2021-03-31,(Other),Renewal,SMS,...,51,NaN,NaN,www.fwd.com.sg/maid-insurance/online-quote/pay...,www.fwd.com.sg/,/maid-insurance/,/online-quote/,/payment-processing?type=renewal&policy=356436735,Maid Insurance Online Quote | FWD Singapore,PAGE
530777,20210331,701696772471,GA1.3.163.1617103223,4,701696772471-1617183736,2021-03-31 09:42:16,2021-03-31,(Other),Renewal,SMS,...,51,NaN,NaN,www.fwd.com.sg/maid-insurance/online-quote/pay...,www.fwd.com.sg/,/maid-insurance/,/online-quote/,/payment-processing?type=renewal&policy=356436735,Maid Insurance Online Quote | FWD Singapore,PAGE
530778,20210331,701696772471,GA1.3.163.1617103223,4,701696772471-1617183736,2021-03-31 09:42:16,2021-03-31,(Other),Renewal,SMS,...,42,NaN,NaN,www.fwd.com.sg/maid-insurance/renewalConfirmation,www.fwd.com.sg/,/maid-insurance/,/renewalConfirmation,NaN,Maid Insurance Online Quote | FWD Singapore,PAGE
530779,20210331,701696772471,GA1.3.163.1617103223,4,701696772471-1617183736,2021-03-31 09:42:16,2021-03-31,(Other),Renewal,SMS,...,42,NaN,NaN,www.fwd.com.sg/maid-insurance/online-quote/ren...,www.fwd.com.sg/,/maid-insurance/,/online-quote/,/renewPolicy,Maid Insurance Online Quote | FWD Singapore,PAGE


In [14]:
test_variable = df_quotation[df_quotation['full_visitor_id'] == 701696772471]
test_variable

,full_visitor_id,client_id,quotation_id,total_sessions_count,max_visit_number,first_session_date,latest_session_date,first_session_date_time,latest_session_date_time,avg_session_quality_dim,...,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator,quote_applicant_nric,first_policy_issue_date,scv_yor_at_quote_created,scv_first_product_type,scv_issued_nfc_policies,scv_existing_customer_flag
148566,7.016968e+11,GA1.3.163.1617103223,CETP1PS8B,1,4,2021-03-31,2021-03-31,2021-03-31 09:42:16,2021-03-31 09:42:16,52,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
603425,7.016968e+11,GA1.3.163.1617103223,8O3Z1JIIZ,1,4,2021-03-31,2021-03-31,2021-03-31 09:42:16,2021-03-31 09:42:16,52,...,Maid,2021-03-30 17:21:03.247,0.0,1.0,G3329530N,2019-05-17 12:33:53.917,2,Maid,2,1
631507,7.016968e+11,GA1.3.163.1617103223,NaN,4,4,2021-03-30,2021-03-31,2021-03-30 11:20:23,2021-03-31 09:42:16,27,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0


In [84]:
for i in test:
    test['Entry Product'] = "Maid"

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [80]:
test[['full_visitor_id','visit_number','session_date','entrance_page_title','exit_page_title','product_type']]

,full_visitor_id,visit_number,session_date,entrance_page_title,exit_page_title,product_type
0,701696772471,1,2021-03-30,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN
1,701696772471,2,2021-03-31,Maid Insurance Online Quote | FWD Singapore,FWD Online Login Portal | FWD Singapore,NaN
2,701696772471,3,2021-03-31,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN
3,701696772471,4,2021-03-31,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN
4,701696772471,4,2021-03-31,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,Maid
5,701696772471,4,2021-03-31,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN
